### Prims Algorithm

- Given a weighted graph, find the minimum spanning tree

- We previously covered Kruskal's algorithm; Prim's is kind of similar
    - Recall that in Kruskal's, we sort the edges from smallest to largest weight, and add them iteratively to build up the minimum spanning tree UNLESS adding an edge forms a cycle

- In Prim's, we don't sort the edges. 
    - Instead, we start by adding the smallest edge
    - Then, we pick the next smallest edge that is **connected to the vertices already added**
    - So the idea is that we will maintain a tree throughout, and we keep adding edges unless adding an edge forms a cycle (like Kruskal's) 

### Example Walkthrough

- Imagine the following graph

In [15]:
import networkx as nx
G = nx.Graph()
G.add_edge(0, 1, weight=4)
G.add_edge(0, 7, weight=8)
G.add_edge(1, 2, weight=8)
G.add_edge(1, 7, weight=11)
G.add_edge(2, 3, weight=7)
G.add_edge(2, 5, weight=4)
G.add_edge(2, 8, weight=2)
G.add_edge(3, 4, weight=9)
G.add_edge(3, 5, weight=14)
G.add_edge(4, 5, weight=10)
G.add_edge(5, 6, weight=2)
G.add_edge(6, 7, weight=1)
G.add_edge(6, 8, weight=6)
G.add_edge(7, 8, weight=7)
# nx.draw_networkx_edge_labels(G, pos=nx.spring_layout(G))
# nx.draw_networkx(G)

- We represent every edge as a tuple `(from_node, to_node, edge_weight)`

- Sorted edges
    - (6,7,1)
    - (2,8,2)
    - (5,6,2)
    - (0,1,4)
    - (2,5,4)
    - (6,8,6)
    - (2,3,7)
    - (7,8,7)
    - (0,7,8)
    - (1,2,8)
    - (3,4,9)
    - (4,5,10)
    - (1,7,11)
    - (3,5,14)

- Stepping through the algorithm
    - The smallest edge is (6,7,1), add it to graph
        - (6,7)
    - The next smallest edge connected to the graph is (5,6,2)
        - (5,6,7)
    - The next smallest edge connected to the graph is (2,5,4)
        - (2,5,6,7)
    - The next smallest edge connected to the graph is (2,8,2)
        - (2,5,6,7,8)
    - The next smallest edge connected to the graph is (2,3,7)
        - Technically, (6,8,6) is the next smallest. But 6 and 8 are already connected, and adding this will created a cycle
        - (2,3,5,6,7,8)
    - (0,7,8)
        - (0,2,3,5,6,7,8)
    - (0,1,4)
        - (0,1,2,3,5,6,7,8)
    - (3,4,9)
        - (0,1,2,3,4,5,6,7,8)


- How do we efficiently retrieve the next smallest connected? 
    - This sort of pattern should trigger us to use a min heap!!

### Code Implementation

In [3]:
from collections import defaultdict

inputs = [
    (6,7,1), (2,8,2),(5,6,2),(0,1,4),(2,5,4),(6,8,6),
    (2,3,7),(7,8,7),(0,7,8),(1,2,8),(3,4,9),(4,5,10),
    (1,7,11),(3,5,14)
]

inputs_weight_first = [
    (x[2], x[0], x[1]) for x in inputs
]

def make_adjacency_map(inputs_weight_first: list[tuple[int, int, int]]) -> dict[int, list[tuple[int, int]]]:
    res = defaultdict(list)
    for (e,f,t) in inputs_weight_first:
        res[f].append((t,e))
        res[t].append((f,e))
    return res

# make_adjacency_map(inputs_weight_first)
# inputs_weight_first;

In [4]:
import heapq

def prims_algorithm(inputs_weight_first: list[tuple[int, int, int]]):
    adj_map: dict[int, list[tuple[int, int]]] = make_adjacency_map(inputs_weight_first)
    priority_queue = [(0,0,0)]
    visited = set()
    edges = []
    spanning_tree_weight = []
        
    while priority_queue:
        # print('='*50)
        # print(f"{priority_queue=}")
        # print(f"{edges=}")
        # print(f"{visited=}")

        e, f,t = heapq.heappop(priority_queue)
        # print(f'{e=}, {f=}, {t=}')
        if t in visited:
            continue
        
        edges.append((f,t))
        visited.add(t)
        spanning_tree_weight.append(e)

        for neighbour, weight in adj_map.get(t, []):
            if neighbour not in visited:
                heapq.heappush(priority_queue, (weight, t, neighbour))
        
        # print(f"{priority_queue=}")
        # print(f"{edges=}")
        # print(f"{visited=}")
        
    return edges, spanning_tree_weight
    # return spanning_tree_weight

edges, weights = prims_algorithm(inputs_weight_first)
sum(weights)

37

### Time Complexity

- Time complexity
    - We create adj_map in $O(E)$ time
    - In each iteration, we do a heappop and a heappush from the minheap, that stores every edge once
        - Heap Pop
            - We can pop from the min heap in $O(\log E)$ time
            - For $E$ edges, this gives us $O(E \log E)$
        - Heap Push
            - We push each vertex once to heap
            - Since there are $V$ vertices, and heappush takes $O(\log E)$ time, total complexity is $O(V \log E)$
    - Overall, this gives us $O((V+E) \log E)$
    
- Space complexity
    - We create adj_map in $O(E)$ space
    - We create the heap in $O(E)$ space
    - We track visited vertices in $O(V)$ space
    - So overall space complexity is $O(E+V)$